<h1 align = "center"> Spark RDD </h1>

Ověření inicializace Spark contextu.

## Vytvoření RDD

Vytvoření RDD pomocí paralelizace kolekce.

Vytvoření RDD z textového souboru.

Vytvoření nového RDD pomocí transformace existujícího.

## Akce nad RDD

Zjištění počtu řádků v RDD.

Maximální hodnota v RDD.

Minimální hodnota v RDD.

Vypsání statistiky z RDD.

Vrácení všech elementů RDD do kolekce.

Vrácení určitého množství elementů RDD do kolekce.

Vrátit první řádek.

Sečtení RDD.

## Cache

## Key-value pairs RDD - Průměrný věk přátel

Vytvoříme nové RDD ze souboru fakefriends-noheader.csv

Zobrazíme si prvních pět řádků

Vytvoření RDD které obsahuje tuple (věk,počet přátel)

Vytvoříme key-value pair kde klíč bude věk a value bude tuple (celkový počet přátel, počet lidí)

Celkový počet přátel vydělíme počtem lidí.

Vypíšeme výsledky.

# Samostatná práce

Vytvořte RDD ze souboru README.md ve složce LabData.

Vypiš počet řádku v souboru.

Vraťte první řádek.

Vypiš všechny řádky které obsahují slovo Spark